In [6]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

: 

In [7]:
# reading the csv file 

df = pd.read_csv('stud.csv')

In [8]:
#inpsect the dataset

print("dataset overview  :",'\n\n',df.head(5))
print("###################################################################################")
print("###################################################################################")
print("Checking the stats of the dataset:","\n\n",df.describe())
print("###################################################################################")
print("###################################################################################")
print("Dimension of the dataset  :",df.shape)
print("###################################################################################")
print("###################################################################################")
print("Checking for missing values :","\n\n",df.isna().sum())
print("###################################################################################")
print("###################################################################################")
print("Checking for duplicate values :","\n\n",df.duplicated().sum())
print("###################################################################################")
print("###################################################################################")
print("checking for unique values in the dataset :",'\n\n',df.nunique())
print("###################################################################################")
print("###################################################################################")

dataset overview  : 

    gender race_ethnicity parental_level_of_education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test_preparation_course  math_score  reading_score  writing_score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  
###################################################################################
####################################################

In [10]:
print("Categorical in 'gender' column :",end=" ")
print(df['gender'].unique(),'->',len(df['gender'].unique()))

print("Categorical in 'race_ethnicity' column :",end=" ")
print(df['race_ethnicity'].unique(),"->",len(df['race_ethnicity'].unique()))

print("\nCategorical in 'parental_level_of_education' column :",end=" ")
print(df['parental_level_of_education'].unique(),'->',len(df['parental_level_of_education'].unique()))

print("\nCategorical in 'lunch' column :",end=" ")
print(df['lunch'].unique(),'->',len(df['lunch'].unique()))

print("\nCategorical in 'test_preparation_course' column :",end=" ")
print(df['test_preparation_course'].unique(),'->',len(df['test_preparation_course'].unique()))

Categorical in 'gender' column : ['female' 'male'] -> 2
Categorical in 'race_ethnicity' column : ['group B' 'group C' 'group A' 'group D' 'group E'] -> 5

Categorical in 'parental_level_of_education' column : ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school'] -> 6

Categorical in 'lunch' column : ['standard' 'free/reduced'] -> 2

Categorical in 'test_preparation_course' column : ['none' 'completed'] -> 2


In [11]:
df['total_mark'] = df['math_score']+df['reading_score']+df['writing_score']
df['average_mark'] = df['total_mark']/3

##Prepareing the dataset to model training (x/y split)

In [12]:
x = df.drop(columns=['total_mark'])
y = df['total_mark']

In [13]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,average_mark
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667
1,female,group C,some college,standard,completed,69,90,88,82.333333
2,female,group B,master's degree,standard,none,90,95,93,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333
4,male,group C,some college,standard,none,76,78,75,76.333333


In [14]:
num_feature = x.select_dtypes(exclude='object').columns
cat_feature = x.select_dtypes(include='object').columns

# creating column transformer with 3 different transformers

from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

std_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer([
    ("One_hotencoder",oh_transformer,cat_feature),
    ("StandardScaler",std_transformer,num_feature),
]
)

In [16]:
x = preprocessor.fit_transform(x)

In [17]:
x.shape

(1000, 21)

In [18]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

## Evaluation metrics function

In [19]:
def evaluatea_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    msr = mean_squared_error(true,predicted)
    rmsr = np.sqrt(mean_absolute_error(true,predicted))
    r2 = r2_score(true,predicted)

    return mae,msr,rmsr,r2 

In [ ]:
models = {
    'Linear Regression' : LinearRegression(),
    'lasso':Lasso(),
    "ridge":Ridge(),
    "support vector machine":SVR(),
    "Random Forest regressor":RandomForestRegressor(),
    "cat boost regressor": CatBoostRegressor(),
    "XGBoost Regressor":XGBRegressor(),
    "ada Boost regressor":AdaBoostRegressor(),
    "KNieghbour-regression":KNeighborsRegressor(),
    "Desicion Tree":DecisionTreeRegressor()
}

model_list = []
r2_list =[]

for i in range (len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train,y_train)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    # evaluating the model 
    model_train_mae , model_train_msr ,model_train_rmsr,model_train_r2 = evaluatea_model(y_train,y_train_pred)
    model_test_mae ,model_test_msr , model_test_rmsr,model_test_r2 = evaluatea_model(y_test,y_test_pred)

    # adding the model to the model list
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model performance for training set :")
    print("Mean Absolute Error :{:0.4f}".format(model_train_mae))
    print("Mean Square error :{:0.4f}".format(model_train_msr))
    print("Root Mean square error:{:0.4f}",format(model_train_rmsr))
    print("R2_square :{:0.4f}".format(model_train_r2))

    print("**************************************************************")

    print("Model performance for testing set :")
    print("Mean Absolute Error :{:0.4f}".format(model_test_mae))
    print("Mean Square error :{:0.4f}".format(model_test_msr))
    print("Root Mean square error:{:0.4f}",format(model_test_rmsr))
    print("R2_square :{:0.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)




Linear Regression


AttributeError: 'LinearRegression' object has no attribute 'keys'

Lasso()